In [2]:
from selenium import webdriver
import time


WEB_DRIVER_PATH = 'C:\chromedriver.exe'
START_DATE = ['20200901']
END_DATE =   ['20200901']
DINING_NAME = ['코로나 외식']

from bs4 import BeautifulSoup
from urllib import request, parse
from selenium import webdriver
import time
import re
import xlwt

DATE = 0
TITLE = 1
TEXT = 2

print("start")

def make_basic_url(keyword, start, end):
    base_url = 'https://m.search.naver.com/search.naver?display=15&nso=p%3A'
    period = 'from' + start + 'to' + end
    query = '&query=' + parse.quote(keyword)
    end = '&where=m_blog&start='
    final_url = base_url + period + query + end
    return final_url

def get_blog_posting_urls(keyword, start, end, driver):
    basic_url = make_basic_url(keyword, start, end)
    blog_postings = []
    index = 1
    flag = True
    regex_href = r'.*https:\/\/m\.blog\.naver\.com\/(\w*\/\d*)'
    while(flag):
        # index에 해당하는 url
        url = basic_url + str(index)

        driver.get(url)
        html = driver.page_source
        bs = BeautifulSoup(html, 'html5lib')
        links = bs.select('a.api_txt_lines')#.bx a
        for single_link in links:
        # single_link가 https://m.blg.naver.com을 포함하면 그걸 가져오자
            href = re.findall(regex_href, str(single_link))
            if href != None and href !=[]:
                if href in blog_postings:
                    flag = False
                    break;
                else:
                    blog_postings.append(href)
        index += 15
    return blog_postings

def get_element(type, posting_addr, driver):
    url = 'https://m.blog.naver.com/' + posting_addr[0]
    driver.get(url)
    html = driver.page_source.encode('utf-8')
    bs = BeautifulSoup(html, 'html5lib', from_encoding='utf-8')

    switcher = {
        0: get_date,
        1: get_title,
        2: get_text
    }
    return switcher.get(type)(bs)

def get_date(bs):
    date_divs = bs.select('p.blog_date')#.se_date
    date = re.findall(r'(20[\d\s\.\:]*)', str(date_divs))
    try:
        return date[0]
    except IndexError:
        return None

def get_text(bs):
    # 네이버는 에디터에 따라 css selctor가 달라진다
    text_divs1 = bs.select('.se_textView > .se_textarea > span,p')
    text_divs2 = bs.select('.post_ct span')

    if len(text_divs1) > len(text_divs2):
        final_text_div = text_divs1
    else:
        final_text_div = text_divs2

    text_for_blog = ''
    for text in final_text_div:
        text = re.sub(r'(\<.+?\>)', '', str(text))
        if text not in text_for_blog:
            text_for_blog += text
    return text_for_blog

def get_title(bs):
    title_divs = bs.select('span#SE-50f1fb93-d4aa-497f-8419-ec99a87c44f4')#.se_title > .se_textView > .se_textarea
    #span#SE-50f1fb93-d4aa-497f-8419-ec99a87c44f4
    if title_divs == []:
        title_divs = bs.select('.tit_h3')
    for title in title_divs:
        final_title = re.sub(r'(\s\s[\s]+)', '', str(title.text))
        return final_title

def save_xlsx(wb, ws, dining_name, list1, list2, list3, index):
    print('save_xlsx')

    for val1, val2, val3 in zip(list1, list2, list3):
        if len(val3)>32000:
            continue
        #ws.write(index, 0, broad_name)
        ws.write(index, 0, dining_name)
        #ws.write(index, 2, broad_date)
        ws.write(index, 1, val1)
        ws.write(index, 2, val2)
        ws.write(index, 3, val3)
        index += 1
    return index

driver = webdriver.Chrome('./chromedriver')
index = 0
wb = xlwt.Workbook(encoding='utf-8')
ws = wb.add_sheet('naver_blog')
for start_date, end_date, dining_name in zip(START_DATE, END_DATE, DINING_NAME):
    # 키워드, 검색 시작/종료 날짜의 포스팅 url을 가져오기
    blog_posting_urls = get_blog_posting_urls(dining_name, start_date, end_date, driver)
    print(start_date, end_date, dining_name)
    # blog_postings의 date, text, title 가져오기
    dates = []
    titles = []
    texts = []

    for posting_addr in blog_posting_urls:
        date = get_element(DATE, posting_addr, driver)
        dates.append(date)

        text = get_element(TEXT, posting_addr, driver)
        texts.append(text)

        title = get_element(TITLE, posting_addr, driver)
        titles.append(title)

    # XLSX_PATH에 저장하기
    index = save_xlsx(wb, ws, dining_name, dates, titles, texts, index)
    
    wb.save("daily30.xls")

start
20200901 20200901 코로나 외식


KeyboardInterrupt: 

In [4]:
index = save_xlsx(wb, ws, dining_name, dates, titles, texts, index)
    
wb.save("daily30.xls")

save_xlsx
